In [113]:
from tools import *
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from tqdm import tqdm  

In [114]:
graph_json_dir = "graphes_JSON"  
graph_json_files = [graph_json_dir+"/"+f for f in os.listdir(graph_json_dir) if f.endswith('.json')]

all_graphs = []

for path in graph_json_files:
    node_features_df=extract_node_features_from_json_file(path)
    edges_df=extract_mapped_edges_from_json(path)
    target_df=extract_optimal_repartition_from_json(path)

    node_features_tensor, edge_index_tensor, y_target_tensor = prepare_data_for_GNN(node_features_df, edges_df, target_df)
    data = Data(x=node_features_tensor, edge_index=edge_index_tensor, y=y_target_tensor)

    all_graphs.append(data)


print("finished data preparting & loading")

finished data preparting & loading


In [115]:
N=64
filtered_graphs = []
for graph in all_graphs:
    num_classes = len(torch.unique(graph.y))  
    
    if graph.x[0][7] <= N:
        filtered_graphs.append(graph)
        
print(f"Selected {len(filtered_graphs)}/{len(all_graphs)} graphs with <=",N, " classes")

Selected 100/100 graphs with <= 64  classes


In [116]:
from torch.utils.data import random_split

total_graphs = len(filtered_graphs)
train_size = int(0.8 * total_graphs)
val_size = int(0.0 * total_graphs)
test_size = total_graphs - train_size - val_size

train_data, val_data, test_data = random_split(filtered_graphs, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8)
test_loader = DataLoader(test_data, batch_size=4,shuffle=True)

In [117]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class MaskedGraphSAGE(torch.nn.Module):
    def __init__(self, num_features, hidden_dim=32, hidden_dim_2=64, hidden_dim_3=32):
        super().__init__()
        # Feature extraction layers
        self.conv1 = SAGEConv(num_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim_2)
        self.conv3 = SAGEConv(hidden_dim_2, hidden_dim_3)
        
        # Fixed output layer (100 classes)
        self.output_layer = torch.nn.Linear(hidden_dim_3, N)
        
        # Masking layer
        self.mask = None  # Will be set per-graph during forward pass

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Feature extraction
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.01, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.01, training=self.training)
        x = F.relu(self.conv3(x, edge_index))
        
        # Get logits for all 100 classes
        logits = self.output_layer(x)
        
        # Apply mask if provided (sets unavailable classes to -inf)
        if hasattr(data, 'cpu_mask'):
            logits = logits + data.cpu_mask
            
        return logits

In [118]:
import torch

NUM_CLASSES = 100  # Fixed output layer size

def apply_class_mask(data):
    # Find unique classes in the graph (i.e., valid class indices)
    valid_classes = data.y.unique()

    # Initialize mask with -inf everywhere
    mask = torch.full((data.num_nodes, NUM_CLASSES), float('-inf'), device=data.y.device)

    # Set 0 where the class is valid (i.e., available in this graph)
    for cls in valid_classes:
        mask[:, cls] = 0.0

    # Attach mask to the data object
    data.cpu_mask = mask
    return data


In [119]:
device = torch.device('cpu')
model = MaskedGraphSAGE(num_features=5).to(device)  
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)


In [120]:
# Calculate masks for all graphs before training
for data in train_data:
    # Assuming data.x[0,6] contains number of available CPUs (e.g., 4)
    num_available_cpus = int(data.x[0,7].item()) 
    
    # Create mask: 
    # - 0 for available CPUs (first 'num_available_cpus' classes)
    # - -inf for others
    mask = torch.full((N,), -float('inf'))
    mask[:num_available_cpus] = 0
# Should show [0,0,0,0,-inf,-inf,...] for 4 CPUs
    # Attach to graph data
    data.mask = mask.to(device)  # Store on same device as model
    print(f"Sample mask for {num_available_cpus} CPUs: {data.mask[:10]}...") 
    selected_indices = [0, 1, 2, 6,7]
    data.x = data.x[:, selected_indices]

for data in test_data: 
    selected_indices = [0, 1, 2, 6,7]
    data.x = data.x[:, selected_indices]

Sample mask for 8 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf])...
Sample mask for 4 CPUs: tensor([0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf])...
Sample mask for 4 CPUs: tensor([0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf])...
Sample mask for 32 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])...
Sample mask for 32 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])...
Sample mask for 8 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf])...
Sample mask for 8 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf])...
Sample mask for 64 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])...
Sample mask for 8 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf])...
Sample mask for 4 CPUs: tensor([0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf])...
Sample mask for 8 CPUs: tensor([0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf])...
Sample mask for 4 CPUs: tensor([0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf])...
Sample mask for 1

In [121]:
def compute_accuracy(logits, targets):
    correct = 0
    for logit, target in zip(logits, targets):
        probs = F.softmax(logit, dim=-1)
        pred = torch.argmax(probs).item()
        if pred == target.item():
            correct += 1
    return correct / len(targets)

In [122]:

for epoch in range(100):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for data in train_data:
        data = data.to(device)
        optimizer.zero_grad()
        
        out = model(data)
        masked_out = out + data.mask  # Apply precomputed mask
      
        targets = data.y.long()
        acc=compute_accuracy(masked_out,targets)
        loss = F.cross_entropy(masked_out, targets)
        loss.backward()
        optimizer.step()

        _, pred = masked_out.max(dim=1)
        total_correct += (pred == targets).sum().item()
        total_samples += targets.size(0)
        total_loss += loss.item()

    if (epoch % 10 == 0) or (epoch == 99):
        epoch_loss = total_loss / len(train_data)
        epoch_acc = total_correct / total_samples
        print(f'Epoch {epoch+1:3d} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')

Epoch   1 | Loss: 2.6997 | Acc: 0.1198
Epoch  11 | Loss: 2.6177 | Acc: 0.1328
Epoch  21 | Loss: 2.6060 | Acc: 0.1383
Epoch  31 | Loss: 2.5933 | Acc: 0.1486
Epoch  41 | Loss: 2.5724 | Acc: 0.1589
Epoch  51 | Loss: 2.5536 | Acc: 0.1715
Epoch  61 | Loss: 2.5359 | Acc: 0.1835
Epoch  71 | Loss: 2.5205 | Acc: 0.1926
Epoch  81 | Loss: 2.4946 | Acc: 0.2059
Epoch  91 | Loss: 2.4692 | Acc: 0.2126
Epoch 100 | Loss: 2.4595 | Acc: 0.2141


In [123]:
# Test Phase
def evaluate(model, loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            pred = out.argmax(dim=1)
            total_correct += (pred == data.y).sum().item()
            total_samples += data.y.size(0)
    
    print(total_correct,total_samples)
    accuracy = total_correct / total_samples
    return accuracy

# Evaluate on all splits
train_acc = evaluate(model, train_loader)
# val_acc = evaluate(model, val_loader)
test_acc = evaluate(model, test_loader)

print('\nFinal Results:')
print(f'Train Accuracy: {train_acc:.4f}')
# print(f'Val Accuracy: {val_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')


1036 9306
186 2322

Final Results:
Train Accuracy: 0.1113
Test Accuracy: 0.0801
